In [1]:
# Import Libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar100

In [2]:
# Load CIFAR-100 Dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0, 1]
y_train, y_test = tf.keras.utils.to_categorical(y_train), tf.keras.utils.to_categorical(y_test)

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [3]:
# Load Pre-trained ResNet50
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    include_top=False,   # Exclude the final classification layer
    input_shape=(32, 32, 3)  # Input shape for CIFAR-100
)

# Freeze Base Model Layers
base_model.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Add Custom Classification Head
inputs = tf.keras.Input(shape=(32, 32, 3))
x = base_model(inputs, training=False)  # Forward pass
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(100, activation='softmax')(x)
model = models.Model(inputs, outputs)

In [6]:
# Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# Train the Model
history = model.fit(x_train, y_train,
                    validation_data=(x_test, y_test),
                    epochs=10,
                    batch_size=64)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.0092 - loss: 4.6055 - val_accuracy: 0.0100 - val_loss: 4.6052
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.0083 - loss: 4.6055 - val_accuracy: 0.0100 - val_loss: 4.6052
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.0093 - loss: 4.6055 - val_accuracy: 0.0100 - val_loss: 4.6052
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.0106 - loss: 4.6055 - val_accuracy: 0.0100 - val_loss: 4.6052
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.0099 - loss: 4.6055 - val_accuracy: 0.0100 - val_loss: 4.6052
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.0102 - loss: 4.6055 - val_accuracy: 0.0100 - val_loss: 4.6052
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.0090 - loss: 4.6055 - val_accuracy: 0.0100 - val_loss: 4.6052
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.0097 - loss: 4.6055 - val_a

In [8]:
# Evaluate the Model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")

313/313 - 4s - 14ms/step - accuracy: 0.0100 - loss: 4.6052
Test Accuracy: 0.01


In [9]:
# Unfreeze Base Model for Fine-Tuning
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 107s 70ms/step - accuracy: 0.0128 - loss: 5.7559 - val_accuracy: 0.0152 - val_loss: 10.4024
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 38ms/step - accuracy: 0.0271 - loss: 4.6186 - val_accuracy: 0.0654 - val_loss: 4.5148
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.0553 - loss: 4.4190 - val_accuracy: 0.1137 - val_loss: 4.1949
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 38ms/step - accuracy: 0.0923 - loss: 4.2168 - val_accuracy: 0.1612 - val_loss: 3.9359
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 37ms/step - accuracy: 0.1333 - loss: 3.9995 - val_accuracy: 0.1997 - val_loss: 3.6863
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.1748 - loss: 3.7661 - val_accuracy: 0.2445 - val_loss: 3.4591
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 38ms/step - accuracy: 0.2180 - loss: 3.5297 - val_accuracy: 0.2784 - val_loss: 3.2400
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 39ms/step - accuracy: 0.2556 - loss: 3.2948 

In [10]:
# Fine-Tune the Model
history_fine = model.fit(x_train, y_train,
                         validation_data=(x_test, y_test),
                         epochs=100,
                         batch_size=64)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 38ms/step - accuracy: 0.3604 - loss: 2.7144 - val_accuracy: 0.3849 - val_loss: 2.6058
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 40ms/step - accuracy: 0.3904 - loss: 2.5579 - val_accuracy: 0.4023 - val_loss: 2.5171
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 38ms/step - accuracy: 0.4203 - loss: 2.4028 - val_accuracy: 0.4144 - val_loss: 2.4553
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 38ms/step - accuracy: 0.4480 - loss: 2.2738 - val_accuracy: 0.4247 - val_loss: 2.4023
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 39ms/step - accuracy: 0.4640 - loss: 2.1916 - val_accuracy: 0.4341 - val_loss: 2.3504
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 38ms/step - accuracy: 0.4907 - loss: 2.0611 - val_accuracy: 0.4402 - val_loss: 2.3127
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.5056 - loss: 1.9749 - val_accuracy: 0.4489 - val_loss: 2.2926
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 42ms/step - accuracy: 0.5309 - loss: 1

KeyboardInterrupt: 

In [11]:
# Final Evaluation
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"Fine-Tuned Test Accuracy: {test_accuracy:.2f}")

313/313 - 4s - 12ms/step - accuracy: 0.4983 - loss: 3.2304
Fine-Tuned Test Accuracy: 0.50
